In [8]:
from random import sample
import numpy as np

# digits from 1 to 9
digits = np.arange(1,10)

problem = np.array([[8, 0, 6, 2, 0, 9, 0, 1, 5],
                    [9, 5, 0, 3, 0, 8, 7, 0, 0],
                    [4, 0, 0, 1, 0, 0, 3, 0, 9],
                    [7, 9, 3, 4, 8, 5, 0, 0, 0],
                    [0, 0, 0, 7, 0, 0, 0, 0, 4],
                    [0, 8, 4, 6, 0, 3, 9, 5, 7],
                    [0, 0, 0, 0, 3, 4, 2, 0, 0],
                    [3, 0, 9, 0, 0, 0, 1, 0, 8],
                    [6, 0, 7, 0, 2, 0, 0, 0, 3]])

# clues idx is a tuple that contains the coordinates of the clues.
# It can be used for indexing non zero values in the problem
clues_idx = np.nonzero(problem)

# Conversely, gaps_idx is a tuple that contains the coordinates of the zeros. 
# It is used for indexing the values we need to fill in.

gaps_idx = np.nonzero(problem == 0)

# We know that a valid solution is a permutation of the digits from 1 to 9 repeated 9 times. Let us find the digits that
# are missing. digit_count counts, starting with zero the number of digits present in the problem:
digit_count = np.bincount(problem.flatten())

# the number of gaps:
problem_size = digit_count[0]

# The missing values are going to be 9 - the count of digits, the zeros can be ignored:
missing_count = 9 - digit_count

# Let´s create a numpy array with the digits that we need to fill in: 
candidate_digits = np.array([], dtype='int32')


for digit in digits:
    candidate_digits = np.append(candidate_digits, np.repeat(digit, missing_count[digit]))


def is_permutation(elements):
  # returns 0 if row is a permutation of digits from 1 to 9
  return 9 - len(np.unique(elements))

def fitness(solution):
  res = 0
  # check that rows are consistent
  for i in range(9):
    res += is_permutation(solution[i])
  
  # check that columns are consistent
  for j in range(9):
    res += is_permutation(solution[:,j])
  
  # check that blocks are consistent
  for k in range(9):   #      0, 1, 2, 3, 4, 5, 6, 7, 8
    row_index = 3*(k//3)   # 3 * (0, 0, 0, 1, 1, 1, 2, 2, 2)
    col_index = 3*(k%3)    # 3 * (0, 1, 2, 0, 1, 2, 0, 1, 2) 
    res += is_permutation(solution[row_index:row_index + 3, col_index:col_index+3])

  # check consistency with problem definition
  # indices of clues
  res += sum(abs(problem[clues_idx]-solution[clues_idx]))

  # check if solution is complete
  res += np.sum(solution == 0)
  return res


population_size = 500

# our genome consists of shuffling the candidate digits and placing the digits in the 
# gaps
    
genome = np.zeros((population_size, 9, 9))
    
for k in range(population_size):
    # suffle the candidates
    np.random.shuffle(candidate_digits)
    
    #init candidate as problem
    genome[k, :, :] = np.copy(problem)
    #fill in sudoku, by placing the digits where the problem is zero
    np.place(genome[k, :, :], genome[k, :, :] == 0, candidate_digits)

# The simplest genome generation strategy is however to just fill the zeros with digits
# np.place(genome, genome==0, [1, 2, 3, 4, 5, 6, 7, 8, 9])

it = 0
num_iter = 1000
iteration_error = np.zeros(num_iter)
while(it < num_iter):
    # reset the fitness function
    fitness_array = np.zeros(population_size)
     
    for k in range(population_size):       
        # calculate fitness function of candidate k
        fitness_array[k] = fitness(genome[k,:,:])

        
    # Find the fittest!
    fitness_array_sorted_idx = np.argsort(fitness_array)
    fittest = fitness_array_sorted_idx[0:10]

    iteration_error[it] = fitness_array[fittest[0]]
    reset = (it >= 10) and (sum(abs(iteration_error[it-10:it] - iteration_error[it])) == 0)
    if reset:
        
        digit_count = np.bincount(problem.flatten())

        # The missing values are going to be 9 - the count of digits, the zeros can be ignored:
        missing_count = 9 - digit_count

        # Let´s create a numpy array with the digits that we need to fill in: 
        candidate_digits = np.array([], dtype='int32')


        for digit in digits:
            candidate_digits = np.append(candidate_digits, np.repeat(digit, missing_count[digit]))

        genome = np.zeros((population_size, 9, 9))
    
        for k in range(population_size):
            # suffle the candidates
            np.random.shuffle(candidate_digits)
    
            #init candidate as problem
            genome[k, :, :] = np.copy(problem)
            #fill in sudoku, by placing the digits where the problem is zero
            np.place(genome[k, :, :], genome[k, :, :] == 0, candidate_digits)
        
        continue    
    
    sol = genome[fittest[0],:,:]
    print('Iteration: ' + str(it))
    print(iteration_error[it])
    # print(sol)    # We might have found already a solution
    
    if fitness_array[fittest[0]] == 0:
        break
    else:
        # Let us create the next generation
        # take ten survivals
        mutations = np.zeros((population_size,9,9))
        # randomly combine rows of the best solutions
        for k in range(population_size):
            # Init mutation_k as problem
            mutation_k = np.copy(problem)
            
            
            # we select the parent randomly from the survivals
            np.random.shuffle(fittest)
            parent_a = genome[fittest[0],:,:]
            
            
            # we first select some of the values to fill in randomly
            pos_to_fill_in_a = np.random.choice(np.arange(problem_size), 95*problem_size//100, replace=False)
                        
            mutation_idx_a = (gaps_idx[0][pos_to_fill_in_a], gaps_idx[1][pos_to_fill_in_a])
            
            # we fill in the values selecting them from the parent
            mutation_k[mutation_idx_a] = parent_a[mutation_idx_a]
            
            # Now we fill in the rest of the values randomly
            digit_count = np.bincount(mutation_k.flatten())
            
            missing_count = 9 - digit_count
            candidate_digits = np.array([], dtype='int32')
            for digit in digits:
                candidate_digits = np.append(candidate_digits, np.repeat(digit, missing_count[digit]))
            
            np.random.shuffle(candidate_digits)
            np.place(mutation_k, mutation_k == 0, candidate_digits)
            
                        
            mutations[k,:,:] = np.copy(mutation_k)
        
        # copy mutation to genome
        genome = np.copy(mutations)
        
        # We increase the number of iterations
        it+=1

print("Best solution is:")
print(sol)

print(iteration_error)

Iteration: 0
51.0
Iteration: 1
47.0
Iteration: 2
42.0
Iteration: 3
39.0
Iteration: 4
35.0
Iteration: 5
34.0
Iteration: 6
32.0
Iteration: 7
28.0
Iteration: 8
27.0
Iteration: 9
25.0
Iteration: 10
25.0
Iteration: 11
22.0
Iteration: 12
20.0
Iteration: 13
19.0
Iteration: 14
19.0
Iteration: 15
18.0
Iteration: 16
18.0
Iteration: 17
16.0
Iteration: 18
16.0
Iteration: 19
15.0
Iteration: 20
15.0
Iteration: 21
13.0
Iteration: 22
13.0
Iteration: 23
13.0
Iteration: 24
11.0
Iteration: 25
11.0
Iteration: 26
11.0
Iteration: 27
11.0
Iteration: 28
11.0
Iteration: 29
9.0
Iteration: 30
9.0
Iteration: 31
8.0
Iteration: 32
8.0
Iteration: 33
8.0
Iteration: 34
8.0
Iteration: 35
6.0
Iteration: 36
6.0
Iteration: 37
6.0
Iteration: 38
6.0
Iteration: 39
6.0
Iteration: 40
6.0
Iteration: 41
6.0
Iteration: 42
6.0
Iteration: 43
6.0
Iteration: 44
6.0
Iteration: 45
47.0
Iteration: 46
45.0
Iteration: 47
43.0
Iteration: 48
41.0
Iteration: 49
37.0
Iteration: 50
36.0
Iteration: 51
34.0
Iteration: 52
32.0
Iteration: 53
31.0


In [9]:
from random import sample
import numpy as np

# digits from 1 to 9
digits = np.arange(1,10)

problem = np.array([[8, 0, 6, 2, 0, 9, 0, 1, 5],
                    [9, 5, 0, 3, 0, 8, 7, 0, 0],
                    [4, 0, 0, 1, 0, 0, 3, 0, 9],
                    [7, 9, 3, 4, 8, 5, 0, 0, 0],
                    [0, 0, 0, 7, 0, 0, 0, 0, 4],
                    [0, 8, 4, 6, 0, 3, 9, 5, 7],
                    [0, 0, 0, 0, 3, 4, 2, 0, 0],
                    [3, 0, 9, 0, 0, 0, 1, 0, 8],
                    [6, 0, 7, 0, 2, 0, 0, 0, 3]])

# clues idx is a tuple that contains the coordinates of the clues.
# It can be used for indexing non zero values in the problem
clues_idx = np.nonzero(problem)

# Conversely, gaps_idx is a tuple that contains the coordinates of the zeros. 
# It is used for indexing the values we need to fill in.

gaps_idx = np.nonzero(problem == 0)

# We know that a valid solution is a collection of permutation of the digits from 1 to 9 repeated 9 times. Let us find the digits that
# are missing. digit_count counts, starting with zero the number of digits present in the problem:
digit_count = np.bincount(problem.flatten())

# the number of gaps:
problem_size = digit_count[0]

# The missing values are going to be 9 - the count of digits, the zeros can be ignored:
missing_count = 9 - digit_count

# Let´s create a numpy array with the digits that we need to fill in: 
candidate_digits = np.array([], dtype='int32')


for digit in digits:
    candidate_digits = np.append(candidate_digits, np.repeat(digit, missing_count[digit]))

missing_per_column = np.zeros((9,9), dtype='int32')
for j in range(9):
    column_count = np.bincount(problem[:,j])
    column_count_padded = np.zeros(10)
    column_count_padded[0:len(column_count)] = column_count
    missing_values = digits[column_count_padded[1:]==0]  
    missing_per_column[0:len(missing_values), j] = missing_values


def is_permutation(elements):
  # returns 0 if row is a permutation of digits from 1 to 9
  return 9 - len(np.unique(elements))

def fitness(solution):
  res = 0
  # check that rows are consistent
  for i in range(9):
    res += is_permutation(solution[i])
  
  # check that columns are consistent
  for j in range(9):
    res += is_permutation(solution[:,j])
  
  # check that blocks are consistent
  for k in range(9):   #      0, 1, 2, 3, 4, 5, 6, 7, 8
    row_index = 3*(k//3)   # 3 * (0, 0, 0, 1, 1, 1, 2, 2, 2)
    col_index = 3*(k%3)    # 3 * (0, 1, 2, 0, 1, 2, 0, 1, 2) 
    res += is_permutation(solution[row_index:row_index + 3, col_index:col_index+3])

  # check consistency with problem definition
  # indices of clues
  res += sum(abs(problem[clues_idx]-solution[clues_idx]))

  # check if solution is complete
  res += np.sum(solution == 0)
  return res


population_size = 500

# Now our genome consists of shuffling the candidate digits of every row and placing the digits in the 
# gaps
    
genome = np.zeros((population_size, 9, 9))
    
for k in range(population_size):
     
    #init candidate as problem
    genome[k, :, :] = np.copy(problem)
    
    #fill in sudoku, by placing the digits where the problem is zero
    for j in range(9):
        candidate_digits_j = missing_per_column[missing_per_column[:, j]>0,j]
        np.random.shuffle(candidate_digits_j)        
        np.place(genome[k, :, j], genome[k, :, j] == 0, candidate_digits_j)

# The simplest genome generation strategy is however to just fill the zeros with digits
# np.place(genome, genome==0, [1, 2, 3, 4, 5, 6, 7, 8, 9])

it = 0
num_iter = 1000
iteration_error = np.zeros(num_iter)
while(it < num_iter):
    # reset the fitness function
    fitness_array = np.zeros(population_size)
     
    for k in range(population_size):       
        # calculate fitness function of candidate k
        fitness_array[k] = fitness(genome[k,:,:])

        
    # Find the fittest!
    fitness_array_sorted_idx = np.argsort(fitness_array)
    fittest = fitness_array_sorted_idx[0:10]

    iteration_error[it] = fitness_array[fittest[0]]
    reset = (it >= 10) and (sum(abs(iteration_error[it-10:it] - iteration_error[it])) == 0)
    if reset:
        genome = np.zeros((population_size, 9, 9))
    
        for k in range(population_size):
            # init candidate as problem
            genome[k, :, :] = np.copy(problem)
    
            # fill in sudoku, by placing the digits where the problem is zero
            for j in range(9):
                candidate_digits_j = missing_per_column[missing_per_column[:, j]>0,j]
                np.random.shuffle(candidate_digits_j)        
                np.place(genome[k, :, j], genome[k, :, j] == 0, candidate_digits_j)

        continue
    
    sol = genome[fittest[0],:,:]
    
    print('Iteration: ' + str(it))
    print(iteration_error[it])
    # print(sol)
    # We might have found already a solution
    if iteration_error[it] == 0:
        break
    else:
        # Let us create the next generation
        # take ten survivals
        mutations = np.zeros((population_size,9,9))
        
        # randomly combine rows of the best solutions
        for k in range(population_size):
            # Init mutation_k as problem
            mutation_k = np.copy(problem)
            
            
            # we select the parent randomly from the survivals
            np.random.shuffle(fittest)
            parent_a = genome[fittest[0],:,:]
            
            
            # we first select eight columns to fill in randomly
            columns_to_fill_in_a = np.random.choice(np.arange(9), 8, replace=False)
                        
            
            # we fill in the values selecting them from the parent
            mutation_k[:,columns_to_fill_in_a] = parent_a[:,columns_to_fill_in_a]
            
            # Now we create another permutation of the column that is missing
            missing_column_k = np.setdiff1d(np.arange(9), columns_to_fill_in_a)
            for j in missing_column_k:
                candidate_digits_j = missing_per_column[missing_per_column[:, j]>0,j]
                np.random.shuffle(candidate_digits_j)        
                np.place(mutation_k[:, j], mutation_k[:, j] == 0, candidate_digits_j)   
            
                        
            mutations[k,:,:] = np.copy(mutation_k)
        
        # copy mutation to genome
        genome = np.copy(mutations)
        
        # We increase the number of iterations
        it+=1

print("Best solution is:")
print(sol)

print(iteration_error)
    












Iteration: 0
25.0
Iteration: 1
22.0
Iteration: 2
19.0
Iteration: 3
17.0
Iteration: 4
14.0
Iteration: 5
12.0
Iteration: 6
12.0
Iteration: 7
11.0
Iteration: 8
9.0
Iteration: 9
9.0
Iteration: 10
7.0
Iteration: 11
7.0
Iteration: 12
6.0
Iteration: 13
6.0
Iteration: 14
6.0
Iteration: 15
5.0
Iteration: 16
5.0
Iteration: 17
5.0
Iteration: 18
5.0
Iteration: 19
5.0
Iteration: 20
5.0
Iteration: 21
5.0
Iteration: 22
5.0
Iteration: 23
5.0
Iteration: 24
5.0
Iteration: 25
23.0
Iteration: 26
20.0
Iteration: 27
19.0
Iteration: 28
17.0
Iteration: 29
14.0
Iteration: 30
14.0
Iteration: 31
13.0
Iteration: 32
13.0
Iteration: 33
13.0
Iteration: 34
11.0
Iteration: 35
11.0
Iteration: 36
10.0
Iteration: 37
10.0
Iteration: 38
10.0
Iteration: 39
10.0
Iteration: 40
10.0
Iteration: 41
10.0
Iteration: 42
10.0
Iteration: 43
10.0
Iteration: 44
9.0
Iteration: 45
9.0
Iteration: 46
9.0
Iteration: 47
6.0
Iteration: 48
6.0
Iteration: 49
6.0
Iteration: 50
5.0
Iteration: 51
5.0
Iteration: 52
5.0
Iteration: 53
5.0
Iteration: 